In [ ]:
#volume and mean crossovers condition

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web

# Download the list of S&P 500 stocks
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
stocks = sp500.Symbol.to_list()

start_date = "2017-01-01"
end_date = "2023-01-31"

primary_stocks = []
secondary_stocks = []

for stock in stocks:
    try:
        # Get the stock price data from Yahoo Finance
        stock_data = yf.download(stock, start=start_date, end=end_date)

        # Calculate the 1-year moving average
        moving_avg = stock_data["Close"].rolling(window=252).mean()

        # Calculate the crossovers
        crossover = np.where(stock_data["Close"] > moving_avg, 1, -1)
        crossover = np.sign(np.diff(crossover))

        # Count the number of positive and negative crossovers
        num_positive_crossovers = len(np.where(crossover > 0)[0])
        num_negative_crossovers = len(np.where(crossover < 0)[0])

        # Determine the trend if the average volume is greater than 1000000
        if stock_data["Volume"].mean() > 1000000:
            if num_positive_crossovers > num_negative_crossovers:
                primary_stocks.append(stock)
            else:
                secondary_stocks.append(stock)
        else:
            secondary_stocks.append(stock)
    except Exception:
        pass

print("Primary stocks:")
print(primary_stocks)

print("Secondary stocks:")
print(secondary_stocks)


In [ ]:
primary_stocks

In [ ]:
#sharpe ratio condition

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from tabulate import tabulate

def std_dev(stocks, start_date, end_date):
    data = yf.download(stocks, start=start_date, end=end_date)['Adj Close']
    ret = data.pct_change()
    
    # Select only the desired stocks
    ret = ret[stocks]
    
    # Calculate the standard deviation of daily percentage changes
    std = ret.std() * np.sqrt(252)
    results = pd.DataFrame({'Standard Deviation (%)': round(std * 100, 4)})
    results.index.name = 'Stock'
    return results.sort_values('Standard Deviation (%)', ascending=False)

# Define the start and end dates for the analysis
start_date = '2017-01-01'
end_date = '2022-12-31'

# Call the std_dev function for the primary_stocks list and sort the results by standard deviation
results = std_dev(primary_stocks, start_date, end_date).sort_values('Standard Deviation (%)', ascending=False)

# Split the sorted results into 9 equal sublists based on standard deviation and name the stock lists
n = 7
chunk_size = len(results) // n
stocks_lists = [results.iloc[i:i+chunk_size].index.tolist() for i in range(0, len(results), chunk_size)]
for i, stocks in enumerate(stocks_lists):
    globals()['stock_list_'+str(i+1)] = stocks

# Call the std_dev function for each sublist of stocks and print the results with stock list names
for i, stocks in enumerate(stocks_lists):
    print(f'Stocks List {i+1}: {globals()["stock_list_"+str(i+1)]}')
    results = std_dev(stocks, start_date, end_date)
    print(tabulate(results, headers='keys', tablefmt='psql'))
    print()


In [ ]:
stock_list_1

In [ ]:
import pandas as pd
import yfinance as yf

def moving_avg(stock_list):
    # Define the start and end dates for the analysis
    start_date = '2022-01-01'
    end_date = '2022-12-31'

    # Download the historical data for the selected stocks
    data = yf.download(stock_list, start=start_date, end=end_date)['Adj Close']

    # Calculate the moving averages for each stock
    moving_avgs = data.rolling(window=20).mean().tail(1).transpose()
    moving_avgs = pd.concat([moving_avgs, data.rolling(window=50).mean().tail(1).transpose()], axis=1)
    moving_avgs = pd.concat([moving_avgs, data.rolling(window=200).mean().tail(1).transpose()], axis=1)

    # Rename the columns
    moving_avgs.columns = ['20-day MA', '50-day MA', '200-day MA']

    # Get the current price of each stock based on the 'Adj Close' column
    current_prices = data.tail(1).transpose()
    current_prices.columns = ['Close']

    # Create empty lists to store the stocks that are greater than the respective moving averages
    new_20_list = []
    new_50_list = []
    new_200_list = []

    # Iterate over each stock
    for stock in stock_list:
        # Check if the current price is greater than the 20-day moving average
        if current_prices.loc[stock, 'Close'] > moving_avgs.loc[stock, '20-day MA']:
            new_20_list.append(stock)

        # Check if the current price is greater than the 50-day moving average
        if current_prices.loc[stock, 'Close'] > moving_avgs.loc[stock, '50-day MA']:
            new_50_list.append(stock)

        # Check if the current price is greater than the 200-day moving average
        if current_prices.loc[stock, 'Close'] > moving_avgs.loc[stock, '200-day MA']:
            new_200_list.append(stock)

    # Return the stocks that are greater than the respective moving averages
    return new_20_list, new_50_list, new_200_list
    



In [ ]:
new_moving_average = moving_avg(stock_list_7)
new_20_list = new_moving_average[0]
print(new_20_list)

new_50_list = new_moving_average[1]
print(new_50_list)

new_200_list = new_moving_average[2]
print(new_200_list)

In [ ]:
import pandas as pd
import yfinance as yf

# Define the start and end dates for the analysis
start_date = '2017-01-01'
end_date = '2022-12-31'

# Define the risk-free rate
rf_rate = 0.02

def calculate_alphas(moving_avg_list):
    # Download the historical data for the selected stock list
    data = yf.download(moving_avg_list, start=start_date, end=end_date)['Adj Close']

    # Calculate the daily returns for each stock
    returns = data.pct_change().dropna()

    # Download the historical data for the market index (e.g., S&P 500)
    market = yf.download('^GSPC', start=start_date, end=end_date)['Adj Close']
    market_returns = market.pct_change().dropna()

    # Calculate the beta and alpha for each stock
    alphas = {}
    for stock in returns.columns:
        cov = returns[stock].cov(market_returns)
        var = market_returns.var()
        beta = cov / var
        rf_adjusted = (1 + rf_rate) ** (1/365) - 1
        alpha = returns[stock].mean() - rf_adjusted - beta * (market_returns.mean() - rf_adjusted)
        alphas[stock] = alpha

    # Sort the stocks by alpha in descending order
    results = pd.Series(alphas).sort_values(ascending=False)

    # Print the results
    print("Stocks sorted by alpha for " + str(moving_avg_list) + ":")
    print(results)

In [ ]:
calculate_alphas(new_200_list)